[View in Colaboratory](https://colab.research.google.com/github/NehruNidamanuru/Deep-learning/blob/master/Lstm_Attention_textclassification.ipynb)

In [88]:
import os
os.chdir("/content/drive/")
!ls

2014-Nehru-Java-68.docx
2018-07-26-13-47-54-515_1532593074515_XXXPN5331X_Acknowledgement.zip
alg 2017_12_07_07_06_54.mp3
alg2017_12_08_07_13_43.mp3
alg2017_12_11_07_16_22.mp3
alg2017_12_12_07_16_00.mp3
alg2017_12_13_07_11_54.mp3
Amar_Mainframes.docx
anomoly.pkl
AP5+PreTunnel-MOTION-1526577006112-2018-137-17-10-02.68.mp4
bottlenck_model_cropped_vgg16_2class.h5
bottlenck_model_vgg_3_class_svm.h5
bottlenck_model_vgg_3_class_svm_small.h5
Capturerrb.jpg
Colab Notebooks
Data (6adc3e0f).rar
Data.rar
Deeplearning
drive
DSC_0053 (3e8ecf81).JPG
DSC_0053.JPG
DSC_0054 (603618a3).JPG
DSC_0054.JPG
DSC_0055 (3fddaa93).JPG
DSC_0055.JPG
DSC_0056 (f80bbf40).JPG
DSC_0056.JPG
DSC_0057 (091332e3).JPG
DSC_0057.JPG
DSC_0058 (07c667ca).JPG
DSC_0058.JPG
DSC_0059 (600f059c).JPG
DSC_0059.JPG
DSC_0060 (c60e80b8).JPG
DSC_0060.JPG
DSC_0061 (9ee580f7).JPG
DSC_0061.JPG
DSC_0062 (10f61870).JPG
DSC_0062.JPG
DSC_0063 (310db53e).JPG
DSC_0063.JPG
DSC_0064 (611156ec).JPG
DSC_0064.JPG
DSC_0065 (00521816).JPG
DSC_0065.JPG
DS

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
import os
from sklearn.model_selection import train_test_split

from keras.layers import concatenate
from sklearn.metrics import mean_squared_error
import nltk



import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from nltk import word_tokenize          
from nltk.corpus import stopwords
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers import Input, Bidirectional,TimeDistributed
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint


In [0]:
import pandas as pd
data_file = pd.read_csv("incidents_wo_502196531.csv")

In [4]:
data_file.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25263 entries, 0 to 25262
Data columns (total 16 columns):
number               25263 non-null object
priority             25263 non-null object
assigned_to          25239 non-null object
assignment_group     25147 non-null object
sys_created_on       25263 non-null object
state                25263 non-null object
cmdb_ci              25017 non-null object
incident_state       25263 non-null object
category             25263 non-null object
subcategory          551 non-null object
short_description    25242 non-null object
description          16137 non-null object
contact_type         25263 non-null object
close_code           25262 non-null object
close_notes          25262 non-null object
sys_created_by       25263 non-null object
dtypes: object(16)
memory usage: 3.1+ MB


In [5]:
data_file=data_file.dropna(subset=['cmdb_ci','short_description'])
data_file.shape
data_file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25017 entries, 0 to 25262
Data columns (total 16 columns):
number               25017 non-null object
priority             25017 non-null object
assigned_to          25017 non-null object
assignment_group     25017 non-null object
sys_created_on       25017 non-null object
state                25017 non-null object
cmdb_ci              25017 non-null object
incident_state       25017 non-null object
category             25017 non-null object
subcategory          551 non-null object
short_description    25017 non-null object
description          16137 non-null object
contact_type         25017 non-null object
close_code           25016 non-null object
close_notes          25016 non-null object
sys_created_by       25017 non-null object
dtypes: object(16)
memory usage: 3.2+ MB


In [6]:
vc = data_file['cmdb_ci'].value_counts()
u  = [i not in set(vc[vc<=100].index) for i in data_file['cmdb_ci']]
data_file = data_file[u]
data_file.info()
list1 = list(data_file['cmdb_ci'].unique())
len(list1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20227 entries, 0 to 25258
Data columns (total 16 columns):
number               20227 non-null object
priority             20227 non-null object
assigned_to          20227 non-null object
assignment_group     20227 non-null object
sys_created_on       20227 non-null object
state                20227 non-null object
cmdb_ci              20227 non-null object
incident_state       20227 non-null object
category             20227 non-null object
subcategory          379 non-null object
short_description    20227 non-null object
description          12642 non-null object
contact_type         20227 non-null object
close_code           20227 non-null object
close_notes          20227 non-null object
sys_created_by       20227 non-null object
dtypes: object(16)
memory usage: 2.6+ MB


40

In [0]:
def buildVocabulary(sentence):
    text = list(set(sentence))
    tokenizer = Tokenizer(lower=False, split=' ')
    tokenizer.fit_on_texts(text)
    return tokenizer

In [0]:
def cleanSentence(sentence):
     sentence_clean= re.sub("[^a-zA-Z0-9]"," ", str(sentence))
     sentence_clean = sentence_clean.lower()
     tokens = word_tokenize(sentence_clean)
     stop_words = set(stopwords.words("english"))
     sentence_clean_words = [w for w in tokens if not w in stop_words]
     return ' '.join(sentence_clean_words)

In [11]:
nltk.download('punkt')
nltk.download('stopwords')

#clean the sentence 
data_file['clean_short_description'] = list(map(cleanSentence, data_file['short_description']))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
corpus_text1 = '\n'.join(data_file['clean_short_description'])
sentences = corpus_text1.split('\n')

In [0]:
tokenizer = buildVocabulary(sentences)

In [0]:
vocab_size = len(tokenizer.word_index) + 1

In [16]:
print(vocab_size)

12483


In [17]:
data_file['word_count'] = data_file['clean_short_description'].apply(lambda x: len(str(x).split(" ")))
#incidents_data['word_count'] = incidents_data['short_description'].apply(lambda x: len(str(x).split(" ")))
data_file[['clean_short_description','word_count']].head(11)

,clean_short_description,word_count
0,outlook crash,2
1,csrc pd98a,2
2,irn wont recieve,3
3,csrc svsbpl02,2
4,csrc saqcll51,2
5,irn recieve,2
6,lou t0317483 error,3
7,csrc job sub01702,3
8,triggered pod oracle printed yet orders needin...,12
9,user unable connect vpn,4


In [18]:
d=data_file["word_count"]
d.value_counts()[:50].index

Int64Index([ 2,  3,  4,  5,  6,  7,  8,  9, 11, 10, 12, 13, 14,  1, 15, 16, 17,
            18, 19, 20, 21, 24, 22, 23, 30, 28, 26, 27, 25, 32, 33, 38, 29, 34,
            37, 31, 35, 36, 46, 40, 44, 42, 43, 50, 47, 45, 56, 54, 39, 80],
           dtype='int64')

In [0]:
def getTrainSequences(sentence, tokenizer,seq_maxlen):
    sent1 = tokenizer.texts_to_sequences(sentence)
    #sent_maxlen = max([len(s) for s in sent1])
    #seq_maxlen = max([sent_maxlen])
    return np.array(pad_sequences(sent1, maxlen=seq_maxlen))

In [0]:
#get sequences for each sentence pair tuple
train_data_need = getTrainSequences(sentences, tokenizer,seq_maxlen=50)

In [21]:
train_data_need.shape

(20227, 50)

In [0]:
sentences_w2c = [line.split(' ') for line in sentences]

In [0]:
from gensim.models import Word2Vec

model_w2v = Word2Vec(sentences_w2c, size=100, window=5, min_count=0, workers=4)
vectors = model_w2v.wv
#vectors.save("custom_w2v_vec.bin")

In [25]:
print(len(list(vectors.vocab)))

12483


In [0]:
word_embed_size = 100

In [0]:
def getEmbeddingWeightMatrix_w2vec(w2vec_vectors,word2idx):    
    embedding_matrix = np.zeros((len(word2idx)+1, word_embed_size))
    for word, i in word2idx.items():
        embedding_vector = w2vec_vectors[word]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [0]:
#get embedding layer weight matrix
embedding_weight_matrix = getEmbeddingWeightMatrix_w2vec(vectors, tokenizer.word_index)

In [29]:
embedding_weight_matrix.shape

(12483, 100)

In [30]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
data_file["cmdb_ci_code"] = lb_make.fit_transform(data_file.cmdb_ci.astype(str))
data_file[["cmdb_ci", "cmdb_ci_code"]].head(11)

,cmdb_ci,cmdb_ci_code
0,Software - Microsoft Outlook,32
1,CSRC ABEND,7
2,Oracle EBS R12 Core - Inventory,24
3,CSRC ABEND,7
4,CSRC ABEND,7
5,Oracle EBS R12 Core - Inventory,24
6,Oracle EBS R12 Core - WSH-Shipping,27
7,CSRC ABEND,7
8,Oracle EBS R12 Core - WSH-Shipping,27
9,vpn service,38


In [70]:
tmp=data_file[["cmdb_ci", "cmdb_ci_code"]]
tmp = tmp.drop_duplicates('cmdb_ci')
tmp.info()
tmp['cmdb_ci_code']=tmp['cmdb_ci_code'].astype(str)
from collections import defaultdict
d = defaultdict(list)
for i, j in zip(tmp.cmdb_ci_code,tmp.cmdb_ci):
    d[i].append(j)
d=dict(d)
print(d)
d['32'][0]
x_train,x_test,y1_train,y1_test=train_test_split(train_data_need,data_file['cmdb_ci_code'], test_size=0.20, random_state=42, stratify=data_file['cmdb_ci_code'])
x_train1,x_val,y1_train1,y1_val=train_test_split(x_train,y1_train, test_size=0.20, random_state=42, stratify=y1_train)
#x_train1,x_test,y1_train1,y1_test=train_test_split(train_data_need,data_file['cmdb_ci_code'], test_size=0.20, random_state=42, stratify=data_file['cmdb_ci_code'])


<class 'pandas.core.frame.DataFrame'>
Int64Index: 40 entries, 0 to 2963
Data columns (total 2 columns):
cmdb_ci         40 non-null object
cmdb_ci_code    40 non-null int64
dtypes: int64(1), object(1)
memory usage: 960.0+ bytes
{'32': ['Software - Microsoft Outlook'], '7': ['CSRC ABEND'], '24': ['Oracle EBS R12 Core - Inventory'], '27': ['Oracle EBS R12 Core - WSH-Shipping'], '38': ['vpn service'], '4': ['AP4 P&A List'], '25': ['Oracle EBS R12 Core - OM'], '10': ['Duo Security'], '17': ['General Question'], '31': ['Software'], '22': ['Office 365'], '37': ['password management'], '13': ['FS – TPTP Laptop Software'], '36': ['hardware-other'], '14': ['FS –TPTP Hardware Depot'], '35': ['hardware'], '5': ['AP5 P&A List'], '3': ['AP3 P&A List'], '16': ['GEA ServiceDesk – Hyderabad'], '15': ['GEA ServiceDesk – Bangalore'], '26': ['Oracle EBS R12 Core - PO Direct'], '8': ['Connectivity'], '6': ['BitLocker'], '39': ['windows 10'], '2': ['AP1 P&A List'], '23': ['Oracle EBS R12 Core'], '33': ['Ul

In [33]:
x_train1.shape

(12944, 50)

In [0]:
from keras.utils import np_utils
y1_train1 = np_utils.to_categorical(y1_train1)
y1_test = np_utils.to_categorical(y1_test)
y1_val=np_utils.to_categorical(y1_val)

In [0]:
y1_train = np_utils.to_categorical(y1_train)


In [0]:
target = np_utils.to_categorical(data_file['cmdb_ci_code'])


In [0]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback
from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed
from keras import backend as K
from keras.models import Model

from sklearn.metrics import roc_auc_score


def dot_product(x, kernel):
    
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

class AttentionWithContext(Layer):    

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [0]:
seq_maxlen=50
word_embed_size = 100

In [0]:
##Model Building on LSTM and  Attention layer
input = Input(shape=(x_train1.shape[1],))

inner = Embedding(input_dim=vocab_size, output_dim=word_embed_size, 
                   input_length=seq_maxlen, weights=[embedding_weight_matrix], 
                   trainable = True) (input)

inner = LSTM(100, return_sequences=True)(inner)
inner = Dropout(0.3)(inner)
# compute importance for each step
#inner = TimeDistributed(Dense(100))(inner) 
words_att = AttentionWithContext()(inner)

inner = Dense(100, activation='relu')(words_att)
#inner = Dense(40, activation='relu')(inner)
output = Dense(y1_train1.shape[1], activation='softmax')(inner)

model_a = Model(inputs = input, outputs = output)

In [0]:
model_a.save('my_lstm_model.h5')

In [40]:
import keras

model_a1 = keras.models.load_model('my_lstm_model.h5', custom_objects={'AttentionWithContext':AttentionWithContext()})

/usr/local/lib/python3.6/dist-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [41]:
print(model_a1.summary())
model_a1.compile(Adam(lr=0.01), 'categorical_crossentropy', metrics=['accuracy'])
#model_a.load_weights('/var/www/projects/nehru/text_classification/code/model_incidents_attention_lstm_word_vec_100d_n1.h5')

save_weights = ModelCheckpoint('model_incidents_attention_lstm_word_vec_100d_new.h5', monitor='val_loss', save_best_only=True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 50, 100)           1248300   
_________________________________________________________________
lstm_1 (LSTM)                (None, 50, 100)           80400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 50, 100)           0         
_________________________________________________________________
attention_with_context_1 (At (None, 100)               10200     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 40)                4040      
Total para

In [0]:
from keras.models import model_from_json
model_json = model_a1.to_json()
with open("model_incident_classification_lstm_attention.json", "w") as json_file:
    json_file.write(model_json) 

In [43]:
history = model_a1.fit(x=x_train1, y=y1_train1, batch_size=32,
          epochs=30, validation_data=(x_val, y1_val), callbacks=[save_weights])


Train on 12944 samples, validate on 3237 samples
Epoch 1/30
12944/12944 [==============================] - 45s 3ms/step - loss: 1.5656 - acc: 0.5745 - val_loss: 1.1616 - val_acc: 0.6691
Epoch 2/30
12944/12944 [==============================] - 44s 3ms/step - loss: 0.9809 - acc: 0.7073 - val_loss: 1.0604 - val_acc: 0.7028
Epoch 3/30
12944/12944 [==============================] - 44s 3ms/step - loss: 0.7407 - acc: 0.7751 - val_loss: 1.0902 - val_acc: 0.7105
Epoch 4/30
12944/12944 [==============================] - 44s 3ms/step - loss: 0.6137 - acc: 0.8123 - val_loss: 1.2120 - val_acc: 0.7000
Epoch 5/30
12944/12944 [==============================] - 45s 3ms/step - loss: 0.5248 - acc: 0.8425 - val_loss: 1.2554 - val_acc: 0.7152
Epoch 6/30
12944/12944 [==============================] - 44s 3ms/step - loss: 0.4756 - acc: 0.8535 - val_loss: 1.3165 - val_acc: 0.7068
Epoch 7/30
12944/12944 [==============================] - 44s 3ms/step - loss: 0.4361 - acc: 0.8644 - val_loss: 1.4475 - val_acc:

In [0]:
predicted_lstm=model_a1.predict(x_test)
predicted_lstm=predicted_lstm.argmax(axis=-1)
y1_test=y1_test.argmax(axis=-1)

In [0]:
accuracy1=np.mean(predicted_lstm==y1_test)

In [46]:
print(accuracy1)

0.6811665842807711


In [47]:
from keras.models import model_from_json

# Model reconstruction from JSON file
with open('model_incident_classification_lstm_attention.json', 'r') as f:
    model1 = model_from_json(f.read())

# Load weights into the Model
model1.load_weights('model_incidents_attention_lstm_word_vec_100d_new.h5')

ValueError: ignored

In [79]:
model_1 = keras.models.load_model('my_lstm_model.h5', custom_objects={'AttentionWithContext':AttentionWithContext()})

/usr/local/lib/python3.6/dist-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [0]:
model_1.load_weights('model_incidents_attention_lstm_word_vec_100d_new.h5')

In [0]:
predicted_lstm1=model_1.predict(x_val)
predicted_lstm1=predicted_lstm1.argmax(axis=-1)
y1_val1=y1_val.argmax(axis=-1)

In [0]:
accuracy1=np.mean(predicted_lstm1==y1_val1)

In [83]:
print(accuracy1)

0.7028112449799196


In [0]:
predicted_lstm2=model_1.predict(train_data_need)
predicted_lstm2=predicted_lstm2.argmax(axis=-1)
target1=target.argmax(axis=-1)

In [85]:
accuracy1=np.mean(predicted_lstm2==target1)
print(accuracy1)

0.7513224897414347


In [0]:
predicted_lstm4=model_1.predict(x_train1)
predicted_lstm4=predicted_lstm4.argmax(axis=-1)
y1_trainn=y1_train1.argmax(axis=-1)

In [87]:
accuracy1=np.mean(predicted_lstm4==y1_trainn)
print(accuracy1)

0.784610630407911


In [0]:
np.savetxt('embedding_weight_matrix_custom_neh.txt', embedding_weight_matrix, fmt='%d')